In [ ]:
pip install langchain langchain_community faiss-gpu langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.0/89.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.7 MB/s eta 0:00:00


In [ ]:
pip install -qU langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('MOON_API_KEY')
os.environ["OPENAI_API_BASE"] = userdata.get('MOON_API_BASE')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

# 定义工具

## 联网搜索工具Tavily

Tavily 是一个专为大型语言模型（LLMs）和检索式生成（RAG）优化的搜索引擎，旨在提供高效、快速且持久的搜索结果。

In [ ]:
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
search = TavilySearchResults(max_results=2)

In [ ]:
search.invoke("what is the weather in SF")

[{'url': 'https://www.wunderground.com/hourly/us/ca/san-francisco/date/2024-07-02',
  'content': 'San Francisco Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the San Francisco area. ... Tuesday 07/02 ...'},
 {'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1719881174, 'localtime': '2024-07-01 17:46'}, 'current': {'last_updated_epoch': 1719881100, 'last_updated': '2024-07-01 17:45', 'temp_c': 23.3, 'temp_f': 73.9, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 15.0, 'wind_kph': 24.1, 'wind_degree': 290, 'wind_dir': 'WNW', 'pressure_mb': 1011.0, 'pressure_in': 29.85, 'precip_mm': 0.01, 'precip_in': 0.0, 'humidity': 50, 'cloud': 0,

## 本地知识库检索工具

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
# 加载网页内容
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
# 分割文本
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
# 生成嵌入向量并存储在FAISS向量数据库中
vector = FAISS.from_documents(documents, OpenAIEmbeddings(openai_api_base=userdata.get('OPENAI_API_BASE'),openai_api_key=userdata.get('OPENAI_API_KEY')))
retriever = vector.as_retriever()

In [ ]:
retriever.invoke("how to upload a dataset")[0]

Document(page_content='description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input[\'postfix\'], # Your AI system goes here    data=dataset_name, # The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={      "version": "1.0.0",      "revision_id": "beta"    },)import { Client, Run, Example } from "langsmith";import { evaluate } from "langsmith/evaluation";import { EvaluationResult } from "langsmith/evaluation";co

将retriever封装为一个工具供智能体调用

In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
"langsmith_search",
"Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)


## 创建工具集

In [ ]:
tools = [search, retriever_tool]

# 大模型调用

In [ ]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="moonshot-v1-32k")

简单调用

In [ ]:
from langchain_core.messages import HumanMessage
response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I help you today? If you have any questions or need assistance, please feel free to ask.'

## 模型绑定工具集

In [ ]:
model_with_tools = model.bind_tools(tools)

不调用工具

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")


ContentString: Hello! How can I assist you today? If you have any questions or need help with something, feel free to ask.
ToolCalls: []


调用联网检索工具

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="通过网络搜索，北京天气怎么样？")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': '北京 天气'}, 'id': 'tavily_search_results_json:0'}]


# 创建智能体

In [ ]:
from langgraph.prebuilt import chat_agent_executor
agent_executor = chat_agent_executor.create_tool_calling_executor(model, tools)

In [ ]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})
response["messages"]

[HumanMessage(content='hi!', id='e2e7c3aa-a7eb-48eb-8d29-10ac6e10e804'),
 AIMessage(content='Hello! How can I assist you today? If you have any questions or need help with something, feel free to ask.', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 134, 'total_tokens': 160}, 'model_name': 'moonshot-v1-32k', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-51c68c2d-a53e-4b6e-8f16-f39f414d1403-0', usage_metadata={'input_tokens': 134, 'output_tokens': 26, 'total_tokens': 160})]

## 本地知识库工具调用

In [ ]:
from langchain_core.messages import HumanMessage
response = agent_executor.invoke(
{"messages": [HumanMessage(content="how can langsmith help with testing?")]}
)
response["messages"]

[HumanMessage(content='how can langsmith help with testing?', id='fa0353f1-b8c1-409e-bf9b-ce0cb8987a38'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'langsmith_search:0', 'function': {'arguments': '{\n    "query": "langsmith testing"\n}', 'name': 'langsmith_search'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 140, 'total_tokens': 154}, 'model_name': 'moonshot-v1-32k', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4c939673-df53-43a4-a55b-3d92a36f3ab2-0', tool_calls=[{'name': 'langsmith_search', 'args': {'query': 'langsmith testing'}, 'id': 'langsmith_search:0'}], usage_metadata={'input_tokens': 140, 'output_tokens': 14, 'total_tokens': 154}),
 ToolMessage(content='Skip to main contentGo to API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a p

## 联网检索工具调用

In [ ]:
response = agent_executor.invoke(
{"messages": [HumanMessage(content="通过网络搜索，北京天气怎么样？")]}
)
response["messages"]

[HumanMessage(content='通过网络搜索，北京天气怎么样？', id='1c89663a-04c9-481d-8b99-1dc44ce60da3'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'tavily_search_results_json:0', 'function': {'arguments': '{\n    "query": "北京 天气"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 140, 'total_tokens': 154}, 'model_name': 'moonshot-v1-32k', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ede5d67a-8030-471b-9d5b-ad03d7bd67d2-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '北京 天气'}, 'id': 'tavily_search_results_json:0'}], usage_metadata={'input_tokens': 140, 'output_tokens': 14, 'total_tokens': 154}),
 ToolMessage(content='[{"url": "http://www.weather.com.cn/weather/101010100.shtml", "content": "\\u6d82\\u64e6SPF\\u5927\\u4e8e15\\u3001PA+\\u9632\\u6652\\u62a4\\u80a4\\u54c1\\u3002\\n\\u6e29\\u5dee\\u8f83\\u5927\\uff0c\\u8f8

# 响应返回

## 流式响应

In [ ]:
for chunk in agent_executor.stream(
{"messages": [HumanMessage(content="通过网络搜索，北京当天气温多少度？")]}
):
  print(chunk)
  print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'tavily_search_results_json:0', 'function': {'arguments': '{\n    "query": "北京 当天气温"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 142, 'total_tokens': 157}, 'model_name': 'moonshot-v1-32k', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c27f126f-3609-466d-bc7d-dd96b7d9bc4b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '北京 当天气温'}, 'id': 'tavily_search_results_json:0'}], usage_metadata={'input_tokens': 142, 'output_tokens': 15, 'total_tokens': 157})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "http://www.nmc.cn/publish/forecast/ABJ/beijing.html", "content": "\\u5929\\u6c14\\u9884\\u62a5 . \\u5929\\u6c14\\u516c\\u62a5. \\u6bcf\\u65e5\\u5929\\u6c14\\u63d0\\u793a ... \\u5236\\u4f5c\\u7ef4\\u62a4\\uff1a\\u56fd\\u5b

## 流式令牌

In [ ]:
async for event in agent_executor.astream_events(
{"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
  kind = event["event"]
  if kind == "on_chain_start":
    if (event["name"] == "Agent"):  # 在创建智能体是可以设置名称`.with_config({"run_name": "Agent"})`
      print(f"Starting agent: {event['name']} with input: {event['data'].get('input')}")
    elif kind == "on_chain_end":
      if (event["name"] == "Agent"):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
        print()
        print("--")
        print(f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}")
  if kind == "on_chat_model_stream":
    content = event["data"]["chunk"].content
    if content:
    # 如果模型返回空，说明是需要调用工具，所以只打印不为空的返回
      print(content, end="|")
  elif kind == "on_tool_start":
    print("--")
    print(f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}")
  elif kind == "on_tool_end":
    print(f"Done tool: {event['name']}")
    print(f"Tool output was: {event['data'].get('output')}")
    print("--")


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


--
Starting tool: tavily_search_results_json with inputs: {'query': 'weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1716776372, 'localtime': '2024-05-26 19:19'}, 'current': {'last_updated_epoch': 1716776100, 'last_updated': '2024-05-26 19:15', 'temp_c': 15.6, 'temp_f': 60.1, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 23.0, 'wind_kph': 37.1, 'wind_degree': 280, 'wind_dir': 'W', 'pressure_mb': 1016.0, 'pressure_in': 30.01, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 60, 'cloud': 25, 'feelslike_c': 15.6, 'feelslike_f': 60.1, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 4.0, 'gust_mph': 27.5, 'gust_kph': 44.3}}"}, {'url': 'https://ww

# 记忆功能

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver.from_conn_string(":memory:")

In [ ]:
agent_executor = chat_agent_executor.create_tool_calling_executor(
    model, tools, checkpointer=memory
)

In [ ]:
config = {"configurable": {"thread_id": "abc1234"}}
for chunk in agent_executor.stream(
{"messages": [HumanMessage(content="whats my name?")]}, config
):
  print(chunk)
  print("----")

{'agent': {'messages': [AIMessage(content="I don't have access to your personal information, including your name. If there's anything specific you'd like to know or discuss, feel free to ask!", response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 132, 'total_tokens': 165}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-188500e0-5ae1-4172-83e0-e874e3f7fc73-0')]}}
----


In [ ]:
for chunk in agent_executor.stream(
{"messages": [HumanMessage(content="whats my name?")]}, config
):
  print(chunk)
  print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob! How can I help you, Bob?', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 154, 'total_tokens': 168}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dbb8f134-1927-4c97-90aa-2ed4804c74e7-0')]}}
----
